# Assignment 4

Approximate time needed: 6-7 hours
    
**DLT and Epipolar Lines**

The goal of this assignment is to aid your understanding of the Direct Linear Transform as well as Epipolar geometry.

1. For the first part of the assignment you will find correspondences between 2D-3D points and estimate the P Matrix. You will then be required to estimate Camera Parameters from this P matrix.

2. The second part of this assignment will require you to construct epipolar lines on two corresponding images.

3. The third part of this assignment will require you to use camera intrinsics to estimate the bounding box of a car.

In [9]:
# Import everything
import numpy as np
import cv2 as cv
print(f"Using OpenCV Version: {cv.__version__}")

Using OpenCV Version: 4.5.3


---


## 4.1 DLT

The Direct Linear Transform method for camera calibration (using correspondences)

### Step 1: Generate Correspondences

The first step to perform DLT is to generate correspondences. The cell below opens a new window. Clicking anywhere on the image should give you the pixel location of the image. Once you're done clicking, close the image window. The cell after displays the points you have clicked.

In [2]:
dlt_file_name = "./q1/dlt1.jpeg"

The given code was using matplotlib and was taking long durations to execute. It has been stored in a separate file and a version using [OpenCV](https://docs.opencv.org/4.x/) has been used here. The following are for reference

1. Basic image handling in OpenCV: tutorial [here](https://docs.opencv.org/4.x/db/deb/tutorial_display_image.html)
    1. [cv.imread](https://docs.opencv.org/4.x/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56): Reads an image and stores as numpy.ndarray, shape is (height, width, channels).
    2. [cv.imshow](https://docs.opencv.org/4.x/d7/dfc/group__highgui.html#ga453d42fe4cb60e5723281a89973ee563): View images
2. Mouse event handling in OpenCV: tutorial [here](https://docs.opencv.org/4.x/db/d5b/tutorial_py_mouse_handling.html)
    1. Window management: [cv.namedWindow](https://docs.opencv.org/4.x/d7/dfc/group__highgui.html#ga5afdf8410934fd099df85c75b2e0888b), [cv.resizeWindow](https://docs.opencv.org/4.x/d7/dfc/group__highgui.html#ga9e80e080f7ef33f897e415358aee7f7e), [cv.destroyWindow](https://docs.opencv.org/4.x/d7/dfc/group__highgui.html#ga851ccdd6961022d1d5b4c4f255dbab34)
    2. Assign callback function using [cv.setMouseCallback](https://docs.opencv.org/4.x/d7/dfc/group__highgui.html#ga89e7806b0a616f6f1d502bd8c183ad3e). The function takes five arguments: [event](https://docs.opencv.org/4.x/d0/d90/group__highgui__window__flags.html#ga927593befdddc7e7013602bca9b079b0), x, y (coordinates of operation), [flags](https://docs.opencv.org/4.x/d0/d90/group__highgui__window__flags.html#gaab4dc057947f70058c80626c9f1c25ce) and some additional parameters (can be passed). 


In [13]:
img = cv.imread(dlt_file_name)
img_points = []
_img_col = []

def img_win_event(event, x, y, flags, params):
    if event == cv.EVENT_LBUTTONUP:
        print(f"Clicked on point (x, y): {x}, {y}")
        img_points.append([x, y])
        # Random color to the point (bright colors)
        _img_col.append(tuple(  # OpenCV needs tuple colors
            map(int, np.random.randint(100, 255, 3))))

img_winname = "Image"
cv.namedWindow(img_winname, cv.WINDOW_NORMAL)   # Resizable window
cv.resizeWindow(img_winname, 500, 900)  # Set initial size
cv.setMouseCallback(img_winname, img_win_event) # Attach mouse event

while True:
    cv.imshow(img_winname, img)
    # Show the markers that are currently selected
    for i, ((cx, cy), col) in enumerate(zip(img_points, _img_col)):
        cv.circle(img, (cx, cy), 15, col, -1)
        cv.putText(img, f"{i+1}", (cx, cy-15), 
            cv.FONT_HERSHEY_SIMPLEX, 1.5, col, 2, cv.LINE_AA)
    k = cv.waitKey(1)
    if k == ord('q'):
        break

cv.destroyWindow(img_winname)
img_points = np.array(img_points)

Clicked on point (x, y): 346, 501
Clicked on point (x, y): 127, 582
Clicked on point (x, y): 516, 567
Clicked on point (x, y): 338, 735
Clicked on point (x, y): 320, 215
Clicked on point (x, y): 84, 340
Clicked on point (x, y): 569, 334


In [14]:
# In case the above code fails, close the window (prevent freezing)
cv.destroyAllWindows()

In [22]:
print(f"Image points are (x, y): \n{img_points}")
# Save it to a file
# np.save("./results/q1/image-points.npy", img_points)
# Retrieve from backup
# _img_points = np.load("./results/q1/image-points.npy")

Image points are (x, y): 
[[346 501]
 [127 582]
 [516 567]
 [338 735]
 [320 215]
 [ 84 340]
 [569 334]]


The image points are picked as follows

![Points on cube](./results/q1/cube-points.png)

The image coordinates are as follows (X, Y values)

```txt
array([[346, 501],
       [127, 582],
       [516, 567],
       [338, 735],
       [320, 215],
       [ 84, 340],
       [569, 334]])
```

7 points were chosen to experiment with an extra validation point


After generating pixel locations, you have to generate the corresponding world points. You have the freedom to chose which point you want as origin. You can consider each side as 4 units.

Ensure that the ordering of your points matches the ordering of image points. Keep track of the sequence of corners clicked.

Choosing the origin the point diagonally opposite to point `1` (hidden, innermost corner of the cube). The X axis is along the left edge of that point, Y axis is along the right edge of that point and the Z axis is upwards (towards point `5`). Using this, the point coordinates are derived. A side length is considered to be `4 units`.


In [23]:
world_points = np.array([   # Each index is [X, Y, Z]
    [4, 4, 4],  # Point 1
    [4, 0, 0],  # Point 2
    [0, 4, 0],  # Point 3
    [4, 4, 0],  # Point 4
    [0, 0, 4],  # Point 5
    [4, 0, 4],  # Point 6
    [0, 4, 4],  # Point 7
])

### Step 2: Calculate the Projection Matrix

Calculate the projection matrix $\mathbf{P}$ that relates the points in world frame to the points in the image coordinates.


Now that we have the correspondences, we can estimate the P matrix. Implement the DLT function that returns the P matrix

Quick recap: x_i = P * X_i where (x_i, X_i) is the ith correspondence, and P is a 3x4 dimensional matrix. P is further decomposed as P = K * [ R | T].


In [174]:
# Calculate and return the camera projection matrix P
def calc_P(wpts: np.ndarray, ipts: np.ndarray):
    """
        Calculates the camera projection matrix P, such that x = P * X
        where X is 4, 1 (homogenized world point) and x is 3, 1
        (homogenized image point). P is 3, 4: the camera projection
        matrix. Uses DLT method: At least 6 point correspondences (not
        lying in a plane) are needed.

        Parameters:
        - wpts: np.ndarray  shape: N, 3
            The points in the world coordinate (non-homogenized)
        - ipts: np.ndarray  shape: N, 2
            Non-homogenized point correspondences of the world points
            in the image
        
        Returns:
        - P: np.ndarray     shape: 3, 4
            The camera projection matrix. The norm of the flattened
            matrix (vector) is 1.
    """
    # Number of correspondences
    N = wpts.shape[0]
    # Homogenized correspondences
    X = np.hstack((wpts, np.ones((N, 1))))
    x = np.hstack((ipts, np.ones((N, 1))))
    # Homogenized equations for i: a_i * p = 0
    a_i = lambda i: np.vstack((
        np.hstack((-X[i], np.zeros(4), x[i][0]*X[i])),
        np.hstack((np.zeros(4), -X[i], x[i][1]*X[i]))
    ))
    # Stacked homogenized matrix: A * p = 0
    A = np.vstack([a_i(i) for i in range(N)])
    # -- p = Null space of A --
    # SVD of A
    U, D, Vh = np.linalg.svd(A)
    p = Vh.T[:, -1] # Last column of V
    # Reshape vector to matrix
    P = p.reshape(3, 4)
    return P


Estimate the Camera Projection matrix: $\mathbf{P}$

In [168]:
P = calc_P(world_points[0:6], img_points[0:6])
print(f"Camera projection matrix (3, 4) = \n{P}")

Camera projection matrix (3, 4) = 
[[-9.27397793e-02  6.98502441e-02 -2.31555368e-02  5.69822232e-01]
 [ 2.69970725e-02  3.24010671e-02 -1.20537567e-01  8.03103678e-01]
 [-5.72785424e-05 -3.71895658e-05 -7.54793065e-05  1.79401383e-03]]


In [173]:
# Test the camera projection (verification)
test_points = [1, 3, 7]    # Test points
dev_pix_thres = 10   # Threshold in pixel deviation (10 pixels)
for tp in test_points:
    t = tp - 1
    # Test point (world homogenized coordinates)
    test_X = np.hstack((world_points[t], 1)).T    # Point as 4, 1
    exp_x = np.hstack((img_points[t], 1)).T     # Point as 3, 1
    # Get x through P (P is )
    tx = P @ test_X
    if np.isclose(tx[2], 0):
        print("Point in P2 is at infinity (last value 0)")
    else:
        tx = tx / tx[2] # Normalize it (last element should be 1)
        dev = np.linalg.norm(exp_x - tx)
        if dev < dev_pix_thres:
            print(f"Test {tp}: Deviation of {dev:.3f} pixels")
        else:
            print(f"Test {tp}: Deviation = {dev:.3f}, too high!")

Test 1: Deviation of 0.307 pixels
Test 3: Deviation of 0.189 pixels
Test 7: Deviation of 5.943 pixels


As observed, there is very little deviation even in the 7th correspondence, though it wasn't used to find P. It can be included and the deviation will be even lesser (but let's keep it separate).

### Theory

- Explain how DLT is implemented and show why it works (proof)
- When does DLT fail?

#### DLT Explanation

The **Direct Linear Transform** is a method to extract a homogeneous relation between corresponding points. In the context of camera calibration, where the aim is to estimate the camera extrinsic and intrinsic parameters, it is the process of finding the camera projection matrix, given the point correspondences (pairs of relations, relating the 3D point coordinates in the world with the 2D image coordinates). Here, the projection equation is rearranged into an equation where finding the null space gives the resultant vector directly.


**Implementation**

The above implementation is described here as steps

1. **Correspondences**: The 3D world to 2D image point correspondences are obtained. Manual identification is used here, but this can be done by the help of a depth sensor, and detection algorithms with a camera (in a real setting). Basically, we have the correspondences $\mathbf{X}_i \leftrightarrow \mathbf{x}_i = [ X_i, Y_i, Z_i, 1 ] \leftrightarrow [ x_i, y_i, 1 ]$ (note that the correspondences are homogenized) relating the 3D point in world to point in pixel coordinates. These points are related as $\mathbf{x}_{i_{(3,1)}} = \mathbf{P}_{(3, 4)} \mathbf{X}_{i_{(4, 1)}}$ where the points are homogenized.
2. **Equation Modeling**: The homogeneous equation is created by rearranging the terms (from correspondences).

    $$
    \mathbf{A}_{(2m, 12)} \mathbf{p}_{(12, 1)} = 0 \quad \textup{where}\; \mathbf{p} = \begin{bmatrix}
        \mathbf{P}_{1,:}^\top \\ \mathbf{P}_{2,:}^\top \\ \mathbf{P}_{3,:}^\top
        \end{bmatrix} \; \textup{and} \; \mathbf{A} = \begin{bmatrix}
        \mathbf{a}_{x_1}^\top \\
        \mathbf{a}_{y_1}^\top \\
        \vdots \\
        \mathbf{a}_{x_i}^\top \\
        \mathbf{a}_{y_i}^\top \\
        \vdots \\
        \mathbf{a}_{x_m}^\top \\
        \mathbf{a}_{y_m}^\top \\
        \end{bmatrix} ; \; \begin{bmatrix}
        \mathbf{a}_{x_i}^\top \\
        \mathbf{a}_{y_i}^\top
        \end{bmatrix} = \begin{bmatrix}
        -\mathbf{X}_i^\top & \mathbf{0}^\top & x_i \mathbf{X}_i^\top \\
        \mathbf{0}^\top & -\mathbf{X}_i^\top & y_i \mathbf{X}_i^\top
        \end{bmatrix}
    $$

    The above formulation is true for $m$ correspondences, here $m=6$ is used (6 correspondences). It can be noted that vector $\mathbf{p}$ is created by using the rows of $\mathbf{P}$ and vertically stacking them as columns.

3. **Null Space**: The relation $\mathbf{Ap} = 0$ may not exactly hold true, mostly because of noisy correspondences, measurements, inexactness, numerical instability, etc. It is therefore best to minimize $\left \| \mathbf{Ap} \right \|$ under the constraint $\left \| \mathbf{p} \right \| = 1$. We perform the _Singular Value Decomposition_ of $\mathbf{A}$ and get the last column of matrix $\mathbf{V}$ (right eigenvectors) as $\mathbf{p}$. This result is the closest to the null space of $\mathbf{A}$.

Once we have $\mathbf{p}$, we can convert it back to $\mathbf{P}$ (break into parts of 4 and arrange the parts as rows). We have achieved the camera projection matrix.

**Proof** (why it works)


### Step 3: Analysing P

The next part of this question is to analyse how good our estimate is. Report the reprojection error of your P matrix

In [ ]:
#Get reprojected points
def get_points(P, X):
    pass

#Get reprojection error
def get_err(x, X, P):
    pass

In [ ]:
print("Reprojection Error is:", get_err(x, X, P))

Given the P matrix, we can estimate K, R and T. Write a function that returns K,R,T given P.
Note: 
P = K * [R | T] where K is a 3x3 matrix and [R | T], P are 3x4 matrices

In [ ]:
def ret_RKT(P):
    pass

R, K, T = ret_RKT(P)

#### Verifying the R, K, T Values

Use `np.isclose` to verify the matrix

**Reproject the world points with P matrix you have obtained via DLT and visualize**

### Zhangs method

For this task, use the inbuilt Opencv function to estimate the K matrix of a camera. Use the checkerboard images 5456-5470 in `q1/zhangs` folder for this task. Familiarize yourself with the different parameters for this method.

**Reference**: https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html

---

## 4.2 Epipolar lines

### Task 1

For this task, you have been given two images of the same scene taken from different view-points. You should first estimate the fundamental matrix from these two images.

<img src="./q2/img1.jpg" alt="image 1" width="400"/>
<img src="./q2/img2.jpg" alt="image 2" width="400"/>


Recall that given a point in one image, it's corresponding location in the other image can be found to along a line viz. the epipolar line. The task given to you is to draw the epipolar lines in the second image for each given point in the first image. You have to repeat this for the other image as well. Draw epipolar lines on the first image for the corresponding points in the second image.

The convention used for F is $x'^{T}Fx$ where $x'$ is the location of the point in the second image. For this question you will need to compute the F matrix on your own without using inbuilt functions. 

**Provide a clear explanation on your approach for both this task and the next**

### Task 2

Report the location of the epipoles. Mathematically find the epipoles and verify that the lines intersect at the epipole.

---


## 4.3 Drawing a bounding box around a car and localizing it in the camera frame

You’ve been provided with an image, also taken from a self-driving car, that shows another car in front. The camera has been placed on top of the car, 1.65 m from the ground, and assume the image plane is perfectly perpendicular to the ground. K is provided to you. Your task is to draw a 3D-bounding box around the car in front as shown. Your approach should be to place eight points in the 3D world such that they surround all the corners of the car, then project them onto the image, and connect the projected image points using lines. You might have to apply a small 5° rotation about the vertical axis to align the box perfectly. Rough dimensions of the car - h: 1.38 m, w: 1.51, l: 4.10. (Hint: Fix a point on the ground as your world origin.). Also estimate the approximate translation vector to the mid-point of the two rear wheels of the car in the camera frame.

![Kitti car](./q3/image.png)